**Пробовал использовать Chroma и FAISS - работают плохо и медленно :(**

In [1]:
import json
from usearch.index import Index

from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama
from langchain_ollama import OllamaEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain_community.vectorstores.usearch import USearch
from langchain_community.document_loaders import TextLoader
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain.text_splitter import RecursiveCharacterTextSplitter

НАДО ВЫБРАТЬ ДРУГУЮ МОДЕЛЬ ДЛЯ ЭМБЕДДИНГА И ДЛЯ САМОЙ ГЕНЕРАЦИИ!!!

In [2]:
# llm = Ollama(model="hf.co/bartowski/microsoft_Phi-4-mini-instruct-GGUF:Q8_0", temperature=0.7)
llm = Ollama(model="hf.co/bartowski/phi-4-GGUF:Q5_K_S", temperature=0.7)
embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-small")

C:\Users\merku\AppData\Local\Temp\ipykernel_3708\3526995855.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="hf.co/bartowski/phi-4-GGUF:Q5_K_S", temperature=0.7)


In [3]:
loader = TextLoader("parsed_book_data.txt", encoding="utf-8")
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
)
docs = splitter.split_documents(documents)

In [4]:
docs[0]

Document(metadata={'source': 'parsed_book_data.txt'}, page_content='"\\u003cp\\u003eРоман-эпопея Михаила Шолохова «Тихий Дон» – одно из наиболее значительных, масштабных и талантливых произведений русскоязычной литературы, принесших автору Нобелевскую премию. Действие романа происходит на фоне важнейших событий в истории России первой половины ХХ века – революции и Гражданской войны, поменявших не только древний уклад донского казачества, к которому принадлежит главный герой Григорий Мелехов, но и судьбу, и облик всей страны. В этом грандиозном произведении нашлось место чуть ли не для всего самого увлекательного, что может предложить читателю художественная литература: здесь и великие исторические реалии, и любовные интриги, и описания давно исчезнувших укладов жизни, многочисленные героические и трагические события, созданные с большой художественной силой и мастерством, тем более поразительными, что Михаилу Шолохову на момент создания первой части романа исполнилось чуть больше двад

In [5]:
usearch = USearch.from_documents(docs, embedding_model)

In [6]:
usearch.similarity_search('Кто написал Анну Каренину?')

[Document(metadata={'source': 'parsed_book_data.txt'}, page_content='"Автор - Лев Толстой.\n Название книги - Анна Каренина.\n Описание книги - «Анна Каренина» – одно из величайших произведений мировой литературы. История всепоглощающей страсти замужней женщины и красавца офицера, ради которого она пошла наперекор всему: мнению света, общепринятой морали, своей совести. В романе Лев Толстой смог нарисовать масштабную картину нравов и быта дворянской среды Петербурга и Москвы второй половины XIX века, сочетающую философские размышления авторского alter ego Левина с передовыми в русской литературе психологическими сюжетами, а также сценами из жизни крестьян. «Анна Каренина» – роман о любви, семье, смысле жизни и, конечно же, женской судьбе. Показывая истории нескольких семей, счастливых и несчастных, Толстой рассказывает о тончайших движениях человеческой души: кажется, в этом романе он понял о людях больше, чем они сами о себе могут понять. Издание органично дополняют иллюстрации Ласло 

In [7]:
retriever = usearch.as_retriever()

In [8]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=retriever)

In [10]:
qa.invoke('Кто написал Анну Каренину?')['result']

'Автором романа "Анна Каренина" является Лев Толстой.'

Сохраняем Index

In [11]:
usearch.index.save('usearch/index.usearch')

Сохраняем docstore

In [12]:
with open("usearch/docstore.json", "w", encoding="utf-8") as f:
    json.dump({k: v.model_dump() for k, v in usearch.docstore._dict.items()}, f, ensure_ascii=False, indent=2)

Сохраняем ids

In [13]:
with open("usearch/ids.json", "w", encoding="utf-8") as f:
    json.dump(usearch.ids, f)

Загружаем модель

In [14]:
with open("usearch/docstore.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)

docstore_data = {k: Document(**v) for k, v in raw_data.items()}
docstore = InMemoryDocstore(docstore_data)

with open("usearch/ids.json") as f:
    ids = json.load(f)

index = Index()
index.load('usearch/index.usearch')

In [15]:
usearch = USearch(embedding=embedding_model, index=index, docstore=docstore, ids=ids)

In [16]:
usearch.similarity_search('Кто написал Анну Каренину?')

[Document(metadata={'source': 'parsed_book_data.txt'}, page_content='"Автор - Лев Толстой.\n Название книги - Анна Каренина.\n Описание книги - «Анна Каренина» – одно из величайших произведений мировой литературы. История всепоглощающей страсти замужней женщины и красавца офицера, ради которого она пошла наперекор всему: мнению света, общепринятой морали, своей совести. В романе Лев Толстой смог нарисовать масштабную картину нравов и быта дворянской среды Петербурга и Москвы второй половины XIX века, сочетающую философские размышления авторского alter ego Левина с передовыми в русской литературе психологическими сюжетами, а также сценами из жизни крестьян. «Анна Каренина» – роман о любви, семье, смысле жизни и, конечно же, женской судьбе. Показывая истории нескольких семей, счастливых и несчастных, Толстой рассказывает о тончайших движениях человеческой души: кажется, в этом романе он понял о людях больше, чем они сами о себе могут понять. Издание органично дополняют иллюстрации Ласло 

In [25]:
retriever = usearch.as_retriever()

In [26]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=retriever)

In [27]:
qa.invoke('Кто написал Анну Каренину?')['result']

'Автором романа "Анна Каренина" является Лев Толстой.'